In [2]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
from bempp.api.linalg import lu
bempp.core.opencl_kernels.set_default_device(0,0)

In [24]:
k = 3.25
h = 0.1
deg = 5

d0 = [0,0,-1] # wave's travel direction
p0 = [1,0,0] # polarization 

In [14]:
rotation_angle = np.pi/9
rotation_mat = [[1,0,0],[0, np.cos(rotation_angle), -np.sin(rotation_angle)],[0,np.sin(rotation_angle), np.cos(rotation_angle)]]

In [15]:
origin_grid1 = bempp.api.shapes.sphere(r = 0.2, origin=[-0.3,0,0],h = h)
origin_grid_vert1 = origin_grid1.vertices

grid_vert1 = np.zeros((3, origin_grid_vert1.shape[1]))
for i in range(grid_vert1.shape[1]):
    grid_vert1[:,i] = rotation_mat @ origin_grid_vert1[:,i]
    
grid1 = bempp.api.Grid(grid_vert1, origin_grid1.elements)
#=======================================================================
origin_grid2 = bempp.api.shapes.sphere(r = 0.2, origin=[0.3,0,0],h = h)
origin_grid_vert2 = origin_grid2.vertices

grid_vert2 = np.zeros((3, origin_grid_vert2.shape[1]))
for i in range(grid_vert2.shape[1]):
    grid_vert2[:,i] = rotation_mat @ origin_grid_vert2[:,i]
    
grid2 = bempp.api.Grid(grid_vert2, origin_grid2.elements)
#=======================================================================
grid_combined = bempp.api.grid.grid.union([grid1,grid2])

In [16]:
div_space1 = bempp.api.function_space(grid1, "RWG", 0)
curl_space1 = bempp.api.function_space(grid1, "SNC", 0)

div_space2 = bempp.api.function_space(grid2, "RWG", 0)
curl_space2 = bempp.api.function_space(grid2, "SNC", 0)

div_space_combined = bempp.api.function_space(grid_combined, "RWG", 0)
curl_space_combined = bempp.api.function_space(grid_combined, "SNC", 0)

In [17]:
origin_unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
origin_vert_unit_sphere = origin_unit_sphere.vertices

vert_unit_sphere = np.zeros((3, origin_vert_unit_sphere.shape[1]))
for i in range(vert_unit_sphere.shape[1]):
    vert_unit_sphere[:,i] = rotation_mat@origin_vert_unit_sphere[:,i]
    
unit_sphere = bempp.api.Grid(vert_unit_sphere,origin_unit_sphere.elements)
    
space_unit_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

In [25]:
elec1 = bempp.api.operators.boundary.maxwell.electric_field(div_space1, div_space1, curl_space1, k)
elec_far1 = bempp.api.operators.far_field.maxwell.electric_field(div_space1, vert_unit_sphere, k)

elec2 = bempp.api.operators.boundary.maxwell.electric_field(div_space2, div_space2, curl_space2, k)
elec_far2 = bempp.api.operators.far_field.maxwell.electric_field(div_space2, vert_unit_sphere, k)

elec_combined = bempp.api.operators.boundary.maxwell.electric_field(div_space_combined, div_space_combined, curl_space_combined, k)
elec_far_combined = bempp.api.operators.far_field.maxwell.electric_field(div_space_combined, vert_unit_sphere, k)

mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere, space_unit_sphere, space_unit_sphere).weak_form().A

In [19]:
def sph_b(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)  

def sph_b_dr(q,x):
    r = np.linalg.norm(x)
    return k*(sph_b(q-1,x) - ((q+1)/(k*r))*sph_b(q,x))

def sph(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)
    
def sph_dazi(p, q, x):
    return 1j * p * sph(p, q, x)

def sph_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = sph(p, q, x)
    y3 = sph(-p, q, x)
    
    if abs(p) == q and p >= 0:
        return (p / np.tan(polar)) * y1 
    elif abs(p) != q and p >= 0:
        y2 = sph(p + 1, q, x)
        return (p / np.tan(polar)) * y1 - np.sqrt((q - p) * (q + p + 1)) * np.exp(-1j * azimuth) * y2
    elif abs(p) == q and p < 0:
        return ((-1)**p) * (-p / np.tan(polar)) * y3 * np.exp(1j*2*p*azimuth)
    elif abs(p) != q and p < 0:
        y4 = sph(-p + 1, q, x)
        return  ((-1)**p) * ((-p / np.tan(polar)) * y3 - np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(-1j * azimuth) * y4) * np.exp(1j*2*p*azimuth)

In [20]:
def U_pq(p, q, x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    return [sph_dpolar(p,q,x)*np.cos(polar)*np.cos(azimuth) - sph_dazi(p,q,x)*np.sin(azimuth)/np.sin(polar), 
            sph_dpolar(p,q,x)*np.cos(polar)*np.sin(azimuth) + sph_dazi(p,q,x)*np.cos(azimuth)/np.sin(polar),
            -sph_dpolar(p,q,x)*np.sin(polar)]
def V_pq(p, q, x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    grad_sph = [sph_dpolar(p,q,x)*np.cos(polar)*np.cos(azimuth) - sph_dazi(p,q,x)*np.sin(azimuth)/np.sin(polar), 
                sph_dpolar(p,q,x)*np.cos(polar)*np.sin(azimuth) + sph_dazi(p,q,x)*np.cos(azimuth)/np.sin(polar),
               -sph_dpolar(p,q,x)*np.sin(polar)]
    hat_x = [np.sin(polar)*np.cos(azimuth), np.sin(polar)*np.sin(azimuth), np.cos(polar)]
    
    return np.cross(hat_x, grad_sph)

In [26]:
coeff_S_V_2 = []
coeff_S_U_2 = []
for q in range(1, deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def V_fun_2(x,n,domain_index,result):
            with objmode():
                result[0] = V_pq(p,q,x)[1]
        V_grid_2 = bempp.api.GridFunction(space_unit_sphere,fun = V_fun_2)
        coeff_S_V_2.append(np.conj(V_grid_2.coefficients)* (-k) *((1j)**(q+1))/(q*(q+1)))
        
        @bempp.api.complex_callable
        def U_fun_2(x,n,domain_index,result):
            with objmode():
                result[0] = U_pq(p,q,x)[1]
        U_grid_2 = bempp.api.GridFunction(space_unit_sphere,fun = U_fun_2)
        coeff_S_U_2.append(np.conj(U_grid_2.coefficients)* ((1j)**q)/(q*(q+1)))

In [27]:
%store coeff_S_V_2

Stored 'coeff_S_V_2' (list)


In [28]:
%store coeff_S_U_2

Stored 'coeff_S_U_2' (list)
